# **Walkthrough**

This document provides a detailed walkthrough of how the Rapid IFC-Based Architectural Cost Estimator works internally.


### **Overview of the Estimator**

The estimator is comprised of:

**`main.py`**, which handles user interaction, menu logic, and presentation of results.

**`util.py`**, which performs all IFC-related data extraction, geometry processing, filtering, and cost-calculation functionality.

### **Menu Functions**

It provides menu of functions relevant to cost estimation of supported element types:
1. **Show element count**  
2. **Show element areas**  
3. **Estimate cost**  
4. **Change element price**  
5. **Change element default area**  
6. **Show cost pie chart**  
7. **Quit**

### **Supported Element Types**

**WINDOW** - `IfcWindow`

**DOOR** - `IfcDoor`

**INTERNAL_WALL** - `IfcWall` or `IfcWallStandardCase` that:
- is **NOT** external (`element_is_external` in `util.py`)
- is **NOT** load-bearing (`element_is_load_bearing` in `util.py`)

 **EXTERNAL_WALL** - `IfcWall` or `IfcWallStandardCase` that:
- **IS** external (`element_is_external` in `util.py`)
- is **NOT** load-bearing (`element_is_load_bearing` in `util.py`)

**FLOOR** - `IfcSlab`

**CEILING** - `IfcCovering`

**ROOF** - `IfcRoof`

**CURTAIN_WALL** - `IfcCurtainWall`


## Function Deep-Dive

### **`main.py`** functions

**Option 1) Show element count** 

- Counts how many of each element type the IFC model contains using count_elements from util.py and prints them

In [ ]:
if choice == "1":
    counts = count_elements(model)
    print("\nElement counts:")
    for k,v in counts.items():
        print(f"{k}: {v}")

**Option 2) Show element areas** 

- Calculates total area for each element type using calculate_areas from util.py and prints them

In [ ]:
elif choice == "2":
    areas = calculate_areas(model, default_areas)
    print("\nElement areas (m²):")
    for k,v in areas.items():
        print(f"{k}: {v:.2f}")

**Option 3) Estimate cost**

- Calculates total area for each element type using calculate_areas from `util.py`

- Calculates total cost for each element type using calculated areas and estimate_cost from `util.py`

In [ ]:
elif choice == "3":
    areas = calculate_areas(model, default_areas)
    total_cost = estimate_cost(areas, prices)
    print(f"\nEstimated total cost: {total_cost:,.2f} DKK")

**Option 4) Change element price**

- Allows the user to update the square-meter price of any element type

In [ ]:
elif choice == "4":
    print("\nCurrent prices (DKK/m²):")
    for k,v in prices.items():
        print(f"{k}: {v}")
    key = input("Enter element type to change: ").upper()
    if key in prices:
        prices[key] = float(input("New price: "))
    else:
        print("Invalid type")

**Option 5) Change element default area**

- Allows the user to update fallback areas used when an element has no geometric or quantity data

In [ ]:
elif choice == "5":
    print("\nCurrent default fallback areas:")
    for k,v in default_areas.items():
        print(f"{k}: {v}")
    key = input("Enter element type to change: ").upper()
    if key in default_areas:
        default_areas[key] = float(input("New area: "))
    else:
        print("Invalid type")

**Option 6) Show cost pie chart**

- Calculates total area for each element type using calculate_areas from `util.py`
- Plots and shows cost pie chart using show_cost_pie_chart from `util.py`

In [ ]:
elif choice == "6":
    areas = calculate_areas(model, default_areas)
    show_cost_pie_chart(areas, prices)

**Option 7: Quit program**

- Ends the loop and exits the script

In [ ]:
elif choice == "7":
    print("Quitting...")
    break

### **`util.py`** functions

**`open_ifc`**

- Tries to open the IFC file at the given path using ifcopenshell
- Prints whether loading succeeded and returns the model object or None.

In [ ]:
def open_ifc(path):
    try:
        model = ifcopenshell.open(path)
        print("IFC model loaded successfully.")
        return model
    except:
        print("Failed to open IFC file.")
        return None

**`element_is_external`**

- Determines whether an element is external.
- First checks the IsExternal attribute, then falls back to checking if the name contains “exterior”.
- Returns True if external, False otherwise.

In [ ]:
def element_is_external(elem):
    try:
        if hasattr(elem, "IsExternal") and elem.IsExternal is not None:
            return bool(elem.IsExternal)
    except:
        pass
    name = str(getattr(elem, "Name", "")).lower()
    return "exterior" in name

**`element_is_load_bearing`**

- Checks whether an element is load-bearing.
- Uses the LoadBearing attribute when available.
- If the element is load-bearing, it is excluded from calculations for this estimator.
- Returns True for load-bearing elements, otherwise False.

In [ ]:
ef element_is_load_bearing(elem):
    try:
        if hasattr(elem, "LoadBearing") and elem.LoadBearing is not None:
            return bool(elem.LoadBearing)
    except:
        pass
    return False

**`get_quantity_area`**

- Attempts to extract an area value from the element’s IFC quantity sets.
- Searches for IfcQuantityArea within any attached property sets.
- Returns the area in m² if found, otherwise None.

In [ ]:
def get_quantity_area(elem):
    if not hasattr(elem, "IsDefinedBy"):
        return None

    for rel in elem.IsDefinedBy:
        pset = rel.RelatingPropertyDefinition
        if pset and pset.is_a("IfcElementQuantity"):
            for q in getattr(pset, "Quantities", []):
                if q.is_a("IfcQuantityArea"):
                    try:
                        return float(q.AreaValue)
                    except:
                        continue
    return None

**`compute_projected_area`**

- Computes the projected area of a mesh by flattening its vertices along a chosen axis.
- Calculates the summed area of all triangular faces in 2D space.
- Used as a geometric fallback when no explicit area is provided.

In [ ]:
def compute_projected_area(verts, faces, axis=2):
    verts2d = np.delete(verts, axis, axis=1)
    area = 0.0
    for f in faces:
        v0, v1, v2 = verts2d[f]
        area += abs(np.cross(v1 - v0, v2 - v0)) / 2.0
    return area

**`geom_area`**

- Generates geometric data for an element using ifcopenshell.geom.
- Estimates surface area by projecting the mesh depending on element type (walls vs. horizontal elements).
- Returns the computed area in m² or 0.0 if geometry is unavailable.

In [ ]:
def geom_area(elem, elem_type):
    try:
        shape = ifcopenshell.geom.create_shape(settings, elem)
        geom = shape.geometry
        verts = np.array(geom.verts).reshape((-1, 3))
        faces = np.array(geom.faces).reshape((-1, 3))

        if elem_type in ["INTERNAL_WALL", "EXTERNAL_WALL"]:
            area_xz = compute_projected_area(verts, faces, axis=1)
            area_yz = compute_projected_area(verts, faces, axis=0)
            return max(area_xz, area_yz) * 0.5

        if elem_type in ["CEILING", "ROOF", "FLOOR"]:
            area_xy = compute_projected_area(verts, faces, axis=2)
            return area_xy * 0.5

        return 0.0

    except:
        return 0.0

**`count_elements`**

- Counts occurrences of each supported element type in the IFC model.
- Splits walls into INTERNAL_WALL and EXTERNAL_WALL based on externality.
- Excludes load-bearing walls from the count.
- Returns a dictionary mapping element types to counts.

In [ ]:
def count_elements(model):
    counts = {
        "WINDOW": len(model.by_type("IfcWindow")),
        "DOOR": len(model.by_type("IfcDoor")),
        "INTERNAL_WALL": 0,
        "EXTERNAL_WALL": 0,
        "FLOOR": len(model.by_type("IfcSlab")),
        "CEILING": len(model.by_type("IfcCovering")),
        "ROOF": len(model.by_type("IfcRoof")),
        "CURTAIN_WALL": len(model.by_type("IfcCurtainWall")),
    }

    walls = model.by_type("IfcWall") + model.by_type("IfcWallStandardCase")
    for wall in walls:
        if element_is_load_bearing(wall):
            continue
        if element_is_external(wall):
            counts["EXTERNAL_WALL"] += 1
        else:
            counts["INTERNAL_WALL"] += 1

    return counts

**`calculate_areas`**

- Calculates total surface area for each element type in the model.
- Area priority: (1) IFC quantity area, (2) element dimensions, (3) geometric projection, (4) default fallback.
- Tracks and reports which elements were approximated or defaulted.
- Returns a dictionary mapping element types to total areas in m².

In [ ]:
def calculate_areas(model, default_areas):
    areas = {k: 0.0 for k in default_areas.keys()}
    approximated = {k: 0 for k in default_areas.keys()}
    defaulted = {k: 0 for k in default_areas.keys()}

    def calc_area(elem, elem_type):
        q_area = get_quantity_area(elem)
        if q_area:
            return q_area, False, False

        if elem_type in ["WINDOW", "DOOR"]:
            h = getattr(elem, "OverallHeight", None)
            w = getattr(elem, "OverallWidth", None) or getattr(elem, "OverallLength", None)
            if h and w:
                return float(h) / 1000 * float(w) / 1000, False, False

        g_area = geom_area(elem, elem_type)
        if g_area > 0:
            return g_area, True, False

        return default_areas.get(elem_type, 1.0), False, True

    def add_area(elems, key):
        for e in elems:
            if key in ["INTERNAL_WALL", "EXTERNAL_WALL"] and element_is_load_bearing(e):
                continue
            area, approx, def_used = calc_area(e, key)
            areas[key] += area
            if approx:
                approximated[key] += 1
            if def_used:
                defaulted[key] += 1

    add_area(model.by_type("IfcWindow"), "WINDOW")
    add_area(model.by_type("IfcDoor"), "DOOR")

    walls = model.by_type("IfcWall") + model.by_type("IfcWallStandardCase")
    for w in walls:
        if element_is_load_bearing(w):
            continue
        key = "EXTERNAL_WALL" if element_is_external(w) else "INTERNAL_WALL"
        area, approx, def_used = calc_area(w, key)
        areas[key] += area
        if approx:
            approximated[key] += 1
        if def_used:
            defaulted[key] += 1

    add_area(model.by_type("IfcSlab"), "FLOOR")
    add_area(model.by_type("IfcCovering"), "CEILING")
    add_area(model.by_type("IfcRoof"), "ROOF")
    add_area(model.by_type("IfcCurtainWall"), "CURTAIN_WALL")

    printed = False

    if any(approximated.values()):
        print("\nThe following elements were approximated using geometric projections:")
        for k, c in approximated.items():
            if c > 0:
                print(f"{k.replace('_',' ').title()}: {c}")
        printed = True

    if any(defaulted.values()):
        print("\nThe following elements used default area values:")
        for k, c in defaulted.items():
            if c > 0:
                print(f"{k.replace('_',' ').title()}: {c}")
        printed = True

    if printed:
        input("\nPress Enter to continue...")

    return areas

**`estimate_cost`**

- Computes the total cost by multiplying each element type’s area by its unit price.
- Sums all individual costs and returns the total in DKK.

In [ ]:
def estimate_cost(areas, prices):
    return sum(areas[k] * prices.get(k, 0) for k in areas)

**`show_cost_pie_chart`**

- Generates and displays a pie chart of cost distribution across element types.
- Converts areas and prices into total cost per type.
- Shows the chart using Matplotlib, or prints a warning if no cost data exists.

In [ ]:
def show_cost_pie_chart(areas, prices):
    labels = []
    values = []
    for k in areas:
        cost = areas[k] * prices.get(k, 0)
        if cost > 0:
            labels.append(k.replace("_", " ").title())
            values.append(cost)

    if not values:
        print("No cost data to show.")
        return

    plt.figure(figsize=(7, 7))
    plt.pie(values, labels=labels, autopct="%1.1f%%")
    plt.title("Cost Distribution of Architectural Elements")
    plt.show()